In [1]:
import joblib
#from scipy import spatial
import numpy as np
#from transformers import pipeline, AutoTokenizer, TFPreTrainedModel  
#from tqdm import tqdm
from gensim.models import Word2Vec


df = joblib.load("../fydjob/output/indeed_proc/ip_2021-03-05.joblib")

In [98]:
df.head(5)

,job_title,job_text,company,location,job_info,query_text,source,job_link,tag_language,reviews,job_info_tokenized,job_text_tokenized,job_text_tokenized_titlecase,job_title_tokenized
0,(Junior) Data Engineer (f/m/x),Customlytics ist die führende App Marketing Be...,Customlytics GmbH,Berlin,(Junior) Data Engineer (f/m/x)\nCustomlytics G...,data science,scrape_json,NaN,en,NaN,"[junior, data, engineer, fmx, customlytics, gm...","[customlytics, ist, die, führende, app, market...","[Customlytics, ist, die, führende, App, Market...","[junior, data, engineer, fmx]"
1,,Responsibilities\n\nAs working student (m/f/x)...,Aroundhome,Berlin,Aroundhome6 Bewertungen - Berlin,data science,scrape_json,NaN,en,NaN,"[aroundhome, bewertungen, berlin]","[responsibilities, as, working, student, mfx, ...","[Responsibilities, As, working, student, mfx, ...",[]
2,,Aufgaben\nAls Werkstudent (m/w/d) IT arbeitest...,Aroundhome,Berlin,"Aroundhome6 Bewertungen - Berlin\nTeilzeit, Pr...",data science,scrape_json,NaN,de,NaN,"[aroundhome, bewertungen, berlin, teilzeit, pr...","[aufgaben, als, werkstudent, mwd, it, arbeites...","[Aufgaben, Als, Werkstudent, mwd, IT, arbeites...",[]
3,,Startdatum\n\nAb sofort\n\nDeine Aufgaben\n\nD...,Alexander Thamm GmbH,Berlin,Alexander Thamm GmbH - Berlin\nFestanstellung,data science,scrape_json,NaN,de,NaN,"[alexander, thamm, gmbh, berlin, festanstellung]","[startdatum, ab, sofort, deine, aufgaben, dein...","[Startdatum, Ab, sofort, Deine, Aufgaben, Dein...",[]
4,Full Stack Developer (m/f/d),We’re Phiture: a leading mobile growth consult...,Phiture,BerlinKreuzberg,Full Stack Developer (m/f/d)\nPhiture - Berlin...,data science,scrape_json,NaN,en,NaN,"[full, stack, developer, mfd, phiture, berlink...","[were, phiture, a, leading, mobile, growth, co...","[Were, Phiture, a, leading, mobile, growth, co...","[full, stack, developer, mfd]"


## Baseline Words2vec model

In [2]:
### instanciate model


w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=20,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     )

### building vocab with tokenized words
w2v_model.build_vocab(df["job_text_tokenized"], progress_per=10000) 


###training the model on the dataset
w2v_model.train(df["job_text_tokenized"], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)


### most similar words example
w2v_model.wv.most_similar(["python"])


[('r', 0.9612724781036377),
 ('scala', 0.9477375149726868),
 ('java', 0.9473602771759033),
 ('perl', 0.9447371363639832),
 ('matlab', 0.9395166635513306),
 ('c', 0.9001803398132324),
 ('julia', 0.899307906627655),
 ('cc', 0.8989156484603882),
 ('programing', 0.8947098255157471),
 ('javascala', 0.8921117782592773)]

In [3]:
w2v_model.save("w2v_model_baseline.model")

## Training a Words2vec model with bi-gram parser

### Preprocessing

In [4]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import sent_tokenize
'''
Preprocessing for the Job descriptions in paresed in senteneces.
Modified form the other preprocessing pipeline
'''
def to_lower(text):
    return text.lower()


def remove_number(text):
    text = ''.join(word for word in text if not word.isdigit())
    
    return text


def remove_punctuation_mod(text):
    
    punct = string.punctuation.replace(".","")
    for punctuation in punct:
        text = text.replace(punctuation, '')
    
    return text


def remove_stopwords(text):

    stop_words = set(stopwords.words('english'))
    new_text = []
    for sent in text:
        sent = word_tokenize(sent) 
        sent = [w for w in sent if w not in stop_words and w not in string.punctuation and w]  
        new_text.append(sent)
    return new_text


def lemmatize_words(text):
    
    lemmatizer = WordNetLemmatizer()
    for sentence in text:
        for word in sentence:
            word = lemmatizer.lemmatize(word)
        
    return text



In [5]:
### filter out german offers
df = df[df["tag_language"] == "en"]

In [6]:
### Apply preprocessing to df
df["job_text_sent"]= df["job_text"].apply(to_lower).apply(remove_number)\
                                    .apply(lambda x : x.replace('\n',' '))\
                                    .apply(remove_punctuation_mod)\
                                    .apply(lambda x: sent_tokenize(x))\
                                    .apply(remove_stopwords)\
                                    .apply(lemmatize_words)




In [101]:
df["job_text_sent"][0]

[['customlytics',
  'ist',
  'die',
  'führende',
  'app',
  'marketing',
  'beratungsagentur',
  'aus',
  'berlin'],
 ['wir',
  'bieten',
  'consulting',
  'und',
  'handson',
  'support',
  'rund',
  'um',
  'app',
  'marketing',
  'strategie',
  'produktmanagement',
  'analytics',
  'crm'],
 ['unser',
  'team',
  'erarbeitet',
  'mit',
  'unternehmen',
  'jeder',
  'größe',
  'konzepte',
  'zur',
  'erfolgreichen',
  'vermarktung',
  'von',
  'mobilen',
  'apps'],
 ['dabei',
  'decken',
  'wir',
  'nicht',
  'nur',
  'das',
  'gesamte',
  'spektrum',
  'infrastruktureller',
  'marketingthemen',
  'ab'],
 ['wir',
  'konzipieren',
  'planen',
  'und',
  'steuern',
  'sowohl',
  'das',
  'ui',
  'ux',
  'design',
  'von',
  'mobilen',
  'apps',
  'als',
  'auch',
  'performance',
  'marketing',
  'kampagnen',
  'für',
  'alle',
  'app',
  'verticals'],
 ['über', 'uns', 'unser', 'data', 'team', 'braucht', 'unterstützung'],
 ['du',
  'bist',
  'motiviert',
  'und',
  'von',
  'der',
  'm

### Parse the sentences

In [11]:
### Turn df into list of sentences
sentences = df["job_text_sent"].tolist()

### reduce the nesting of the list to fit the format of the Phrases module
sentence = []
for second in sentences:
    for first in second:
        sentence.append(first)
        

In [12]:
### Train the phraser to detect bi-grams
from gensim.models.phrases import Phrases

phrases = Phrases(sentence, min_count=30, progress_per=10000)
### transform the list of sentences to detect bigrams
sent = []
for phrase in phrases[sentence]:
    sent.append(phrase)

### Word2vec model v2
    

In [13]:
from gensim.models import Word2Vec

w2v_model2 = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     )

### building vocab with tokenized words
w2v_model2.build_vocab(sent, progress_per=10000) 


###training the model on the dataset
w2v_model2.train(sent, total_examples=w2v_model2.corpus_count, epochs=30, report_delay=1)


### most similar words example
w2v_model2.wv.most_similar(["python"])


[('knowledge', 0.9957853555679321),
 ('spark', 0.9938962459564209),
 ('field', 0.9938292503356934),
 ('computer_science', 0.9924174547195435),
 ('cloud', 0.9908705949783325),
 ('engineering', 0.9908539056777954),
 ('infrastructure', 0.990630030632019),
 ('design', 0.9901478290557861),
 ('proficiency', 0.9899214506149292),
 ('java', 0.9878365993499756)]

## Vectorizer

In [19]:
def vectorizer(text):
    '''
    Replace the text with the respective vectors if there are in the model vocabulary
    '''
    new_text = []
    for word in text:
        if word in w2v_model.wv.vocab:
            vector = w2v_model.wv.__getitem__(word)
            new_text.append(vector)
    
    return new_text
    

In [22]:
df["vectorized_jobs"] = df["job_text_tokenized"].apply(vectorizer)

## Quick test for Translation pipeline and saving code for posterity


In [ ]:
### quick test for transformer

from transformers import pipeline

summarizer = pipeline("summarization")

summarizer(df["job_text"][10],min_length=120, max_length=120)

All model checkpoint layers were used when initializing TFT5Model.

All the layers of TFT5Model were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5Model for predictions without further training.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (724 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
### vector extraction from transfomer model



feature_extractor = pipeline("feature-extraction", model = "distilbert-base-cased")

def similarity(s1, s2):
    return  1 - spatial.distance.cosine(feature_extractor(s1)[0][-1], feature_extractor(s2)[0][-1])

def get_features(s):
    return feature_extractor(s)[0][-1]

sentance1 = "no one loves sushi"
sentance2 = "I use java for backend stuff and I'm important"
sentance3 = "I use html and css for be the frontend guy there is"

print(similarity(sentance1, sentance2))
print(similarity(sentance2, sentance3))


# modeling
from sklearn.cluster import KMeans
model  = KMeans(n_clusters=2)
X= np.array([get_features(sentance1),get_features(sentance2),get_features(sentance3)])
model.fit(X)
model.predict(X)
tokens = s.lower().replace('  ',' ').replace('\n',' ').split(' ')
threshold = 0.8
for token in tqdm(set(tokens)):
    if threshold < similarity(token.lower(), 'Skills'.lower()):
        print(token)


In [131]:
import pandas as pd
with open("../fydjob/data/dicts/skills_dict.json") as json_file:
    skills = json.load(json_file)

skills["business"]

['accounting',
 'marketing',
 'sales',
 'distribution',
 'logistic',
 'scm',
 'hr',
 'administration',
 'administrative',
 'management',
 'planning',
 'strategy',
 'strategic',
 'strategical',
 'bd',
 'reporting',
 'report',
 'visualization',
 'visualisation',
 'kpi',
 'business',
 'stakeholder',
 'client',
 'industry',
 'entrepreneurship',
 'entrepreneur',
 'entrepreneurial',
 'consulting',
 'consult',
 'analyst',
 'analyze',
 'analyse',
 'analysis',
 'analytical',
 'analytics',
 'analytic',
 'solution',
 'olap',
 'feasibility',
 'measurable',
 'profitable',
 'commercial',
 'crm',
 'efficiency',
 'advertising',
 'managing',
 'dashboards']

In [10]:
df["job_text_sent"]

0       [[führende, app, marketing, beratungsagentur, ...
1       [[student, mfx, work, collaboration, team, bus...
4       [[phiture, growth, consultancy, closeknit, tea...
5       [[operating], [], [desire], [], [passion], [pu...
6       [[head, finance, home, mission, housing, every...
                              ...                        
1841    [[zalando, fulfillment, controller, perform], ...
1846    [[caya], [manage, process, caya, platform], [c...
1849    [[position, point, contact, architecture, soft...
1902    [[position, point, contact, architecture, soft...
1919    [[industry, market, road, freight, company, ge...
Name: job_text_sent, Length: 839, dtype: object

In [9]:
from nltk.tag import pos_tag

def noun_extractor(text):
    '''
    This function takes a text and returns only the nouns in the text.
    It can be adjusted to any POS by changing the tag below.
    '''
    new_text = []
    for sentence in text:
        tagged_sentence = pos_tag(sentence)
        new_text.append([word for word,tag in tagged_sentence if tag in ("NN")]) #change me!
    return new_text
    

df["job_text_sent"] =df["job_text_sent"].apply(noun_extractor)
              

            
       
        
    


## Word2vec trained exclusivley on nouns

In [26]:
### extract nouns

df["job_text_sent"] = df["job_text_sent"].apply(noun_extractor)

In [9]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=20,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     )

### building vocab with tokenized words
w2v_model.build_vocab(df["job_text_sent"], progress_per=10000) 


###training the model on the dataset
w2v_model.train(df["job_text_sent"], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)


### most similar words example
w2v_model.wv.most_similar(["python"])


TypeError: unhashable type: 'list'